# Camera Calibration

Before we can gather any information from whatever picture, we ought to obtain the parameters of our sensor(s). Which in this case is the camera. 
<br/> Therefore, we'll start with the camera calibration using the _chessboard_ provided at the beggining of the assignment.
<br/>
In short, camera calibration is estimating the parameters of a camera. <br/>
- On the one side, we'll need intrinsic parameters like the focal length (f_x, f_y) and optical centers (c_x, c_y), which will be used to create the camera matrix. Which in turn will be used to remove distortion due to the lenses of a specific camera. Each camera having it's own distortion, meaning that once the camera calibration has been calculated we will be able to use said matrix for future operations using the same camera. </br>
- On the other side, we'll need extrinsic parameters, meaning rotation and translation vectors which translate coordinate points of a 3D point to a coordinate system.
<br/>
<br/>

### Step 1
For starters we'll need to define the size and dimension of our _chessboard_. 
<br/>
It's dimension is defined by the amount of pixels along the width and the lenght of an image, in this case we're working with 1920x1080.
And it's size is defined as the number of intersections between the black and grey squares that make up the _chessboard_, which in this case represents a 7x7, which corresponds to the pattern we are looking for. 

In [1]:
import cv2 as cv
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image


#chessboard dimensions
chessboardSize = (7, 7)
frameSize = (1920, 1080)

### Step 2
Next, we'll need to define the termination criteria, i.e, the "Limit on number of iterations: When the limit on number of iterations or number of function evaluations exceeds a specified value, the iteration is terminated" [^1]. <br/>
For this purpose we'll use the two classes TERM_CRITERIA_EPS and TERM_CRITERIA_MAX_ITER: <br/>
- TERM_CRITERIA_EPS = stop the algorithm iteration if specified accuracy, epsilon, is reached
- TERM_CRITERIA_MAX_ITER = stop the algorithm after the specified number of iterations, max_iter

In [2]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

### Step 3
For simplicity's sake, we suppose that the _chessboard_ is stationary and thus we do not need to consider the _Z_ variables (_Z = 0_).</br>
With this information in mind, we can already prepare  _XY_ coordinates of object points.

In [3]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)

### Step 4
Prepare arrays to store our object points (3D) and image points (2D) and fetch _chessboard_ images with the glob function.

In [10]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

#pc
images = sorted(glob.glob(r'C:\Users\edgar\Desktop\Github\ImageProcessing\lab3&4\chessboards\*.png'))
#laptop
#images = sorted(glob.glob(r'C:\Users\edgar\Documents\GitHub\ImageProcessing\lab3&4\chessboards\*.png'))

### Step 5
Use cv.findChessboardCorners() to find the aforementioned pattern in the _chessboard_. And once the corners have been found, use cv.cornerSubPix() to increase their accuracy.

In [11]:
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
    
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)
        
        # Draw and display the corners
        cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
        
cv.destroyAllWindows()

In [13]:
#Get exif data in order to get focal length. 
exif_img = PIL.Image.open(images[0])

exif_data = {
 PIL.ExifTags.TAGS[k]:v
 for k, v in exif_img._getexif().items()
 if k in PIL.ExifTags.TAGS}

#Get focal length in tuple form
focal_length_exif = exif_data['FocalLength']

#Get focal length in decimal form
focal_length = focal_length_exif[0]/focal_length_exif[1]
np.save("./camera_params/FocalLength", focal_length)

<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1920x1080 at 0x21AC65EF4C0>


AttributeError: 'NoneType' object has no attribute 'items'

### Step 6
Using the cv.calibrateCamera(), we can start the actual camera calibration. This will create the camera matrix, distortion coefficients, rotation and translation vectors etc.

In [18]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

#Save parameters into numpy file
np.save("./lab3&4/cameraParams/ret", ret)
np.save("./lab3&4/cameraParams/mtx", mtx)
np.save("./lab3&4/cameraParams/dist", dist)
np.save("./lab3&4/cameraParams/rvecs", rvecs)
np.save("./lab3&4/cameraParams/tvecs", tvecs)

### Step 7
Using the cv.getOptimalNewCameraMatrix() we can refine the camera matrix based on a free scaling parameter. Alpha being the scaling parameter, we can choose it's value based on our application. For instance, if we need to see all the input pixels, then alpha = 1. Otherwise, we choose an alpha that keeps the intersting part of the image inside the undistorted image. This process allows us to undistort the image

In [19]:
#pc
#img = cv.imread(r'C:\Users\edgar\Desktop\Github\ImageProcessing\lab3&4\chessboards\c4Left.png')
#laptop
img = cv.imread(r'C:\Users\edgar\Documents\GitHub\ImageProcessing\lab3&4\chessboards\c4Left.png')

h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

### Step 8
To achieve the undistortion, we may use two methods for this:
 - Using cv.undistort()
 - Using remapping

For the purposes of this project, we'll use the undistort() method.

In [20]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult.png', dst)

True

[^1]: https://www.sciencedirect.com/topics/engineering/termination-criterion#:~:text=Termination%20Criteria&text=1.,sense%2C%20the%20iteration%20is%20terminated.